In [3]:
def fix_text(text):
    return text.replace("{", "ä").replace("}", "å").replace("|", "ö")

In [4]:
TESTF = "/Users/joregan/Playing/waxholm/scenes_formatted//fp2033/fp2033.6.20.smp.mix"

In [5]:
class FR:
    def __init__(self, text):
        if not text.startswith("FR"):
            raise IOError("Unknown line type (does not begin with 'FR'): " + text)
        parts = text.split("\t")
        if len(parts) == 5:
            self.type = 'B'
        if len(parts) == 4:
            self.type = 'I'
        if len(parts) == 3:
            self.type = 'E'
            if parts[1].strip() != "OK":
                raise IOError("Unexpected line: " + text)
        self.frame = parts[0][2:].strip()
        if len(parts) > 3:
            self.phone_type = parts[1].strip()[0:1]
            self.phone = parts[1].strip()[1:]
            if not parts[2].strip().startswith(">pm "):
                raise IOError("Unexpected line: " + text)
            self.pm_type = parts[2].strip()[4:5]
            self.pm = parts[2].strip()[5:]
        if len(parts) == 5:
            if not parts[3].strip().startswith(">w "):
                raise IOError("Unexpected line: " + text)
            self.word = fix_text(parts[3].strip()[3:])
        if parts[-1].strip().endswith(" sec"):
            self.seconds = parts[-1].strip()[0:-4]

    def __repr__(self):
        parts = []
        parts.append(f"type: {self.type}")
        parts.append(f"frame: {self.frame}")
        if self.type != 'E':
            parts.append(f"phone: {self.phone}")
        if 'word' in self.__dict__:
            parts.append(f"word: {self.word}")
        if 'pm_type' in self.__dict__:
            parts.append(f"pm_type: {self.pm_type}")
        if 'pm' in self.__dict__:
            parts.append(f"pm: {self.pm}")
        parts.append(f"sec: {self.seconds}")
        return f"FR(" + ", ".join(parts) + ")"
        

In [6]:
class Mix():
    def __init__(self, filepath):
        self.fr = []
        with open(filepath) as inpf:
            saw_text = False
            saw_phoneme = False
            saw_labels = False
            for line in inpf.readlines():
                if line.startswith("Waxholm dialog."):
                    self.filepath = line[15:].strip()
                if line.startswith("TEXT:"):
                    saw_text = True
                if saw_text:
                    self.text = fix_text(line.strip())
                    saw_text = False
                if line.startswith("FR "):
                    if saw_labels:
                        saw_labels = False
                    self.fr.append(FR(line))
                if line.startswith("Labels: "):
                    self.labels = line[8:].strip()
                    saw_labels = True
                if saw_labels and line.startswith(" "):
                    self.labels += line.strip()
        

In [7]:
def smp_probe(filename):
    with open(filename, "rb") as f:
        return f.read(9) == b"file=samp"

In [8]:
TEST2 = "/Users/joregan/Playing/waxholm/scenes_formatted/fp2060/fp2060.11.03.smp"

def smp_headers(filename):
    with open(filename, "rb") as f:
        f.seek(0)
        raw_headers = f.read(1024)
        raw_headers = raw_headers.rstrip(b'\x00')
        asc_headers = raw_headers.decode("ascii")
        asc_headers.rstrip('\x00')
        tmp = [a for a in asc_headers.split("\r\n")]
        back = -1
        while abs(back) > len(tmp) + 1:
            if tmp[back] == '=':
                break
            back -= 1
        tmp = tmp[0:back-1]
        return dict(a.split("=") for a in tmp)

smp_headers(TEST2)

{'file': 'samp',
 'msb': 'first',
 'sftot': '16000',
 'nchans': '1',
 'preemph': 'no',
 'view': '-10,10',
 'born': '/o/libhex/ad_da.h25',
 'range': '-6400,4528'}

In [9]:
import struct
def smp_read(filename):
    headers = smp_headers(filename)
    if headers["msb"] == "last":
        SPEC = "<H"
    else:
        SPEC = ">H"
    
    with open(filename, "rb") as f:
        f.seek(1024)
        buf = f.read()
        return [struct.iter_unpack(SPEC, buf)]


In [10]:
a=smp_read(TEST2)

In [6]:
import torch
waveform = torch.tensor(a, dtype=torch.uint16)
waveform = torch.reshape(waveform, (1, waveform.shape[0]))
waveform

In [11]:
import wave
import numpy as np

na = np.array(a, dtype=np.uint16)

with wave.open("sound1.wav", "w") as f:
    # 2 Channels.
    f.setnchannels(1)
    # 2 bytes per sample.
    f.setsampwidth(2)
    f.setframerate(16000)
    f.writeframes(na)

In [1]:
import IPython
IPython.display.Audio("sound1.wav")

In [2]:
na

NameError: name 'na' is not defined